<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Leveraging Apache Spark for Smart Building HVAC Monitoring

**Estimated time needed: 30 minutes**

### Objectives

After completing this lab, you will be able to:

- Explain the distributed architecture of Spark in the context of smart building monitoring
- Simulate real-time sensor data for HVAC systems in a building
- Perform SQL queries to detect critical environmental conditions and calculate average readings
- Determine the aggregated results to the console for immediate insights into room conditions


## Background
Smart Building Solutions, Inc. specializes in optimizing HVAC (heating, ventilation, and air conditioning) systems to enhance comfort and energy efficiency in commercial buildings. By monitoring temperature and humidity levels in real-time across various rooms, the company aims to ensure optimal indoor conditions and preemptively address potential HVAC issues.

With a continuous influx of sensor data, Smart Building Solutions needs to process and analyze this data in real-time to maintain the quality of the indoor environment.

## Data set description
The simulated data set comprises:

`room_id`: Unique identifier for each room (e.g., R001, R002).

`temperature`: Current temperature reading from the sensor (in °C).

`humidity`: Current humidity level reading from the sensor (in %).

`timestamp`: Time when the reading was recorded (automatically generated by Spark).
The data is generated at a rate of 5 rows per second, simulating multiple rooms with various environmental conditions.


## Challenges
Monitoring indoor environmental conditions poses several challenges:

**High data velocity**: Continuous data from multiple sensors can overwhelm traditional systems.

**Need for immediate alerts**: Delays in identifying critical conditions can lead to discomfort or system inefficiencies.

**Need for data aggregation and analysis**: Efficiently aggregating and analyzing real-time data for proactive maintenance and optimization is essential.

## Apache Spark with structured streaming
To address these challenges, Apache Spark is employed for its powerful distributed computing capabilities, enabling real-time data processing and analytics.


In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.clustering import KMeans


from pyspark.sql import SparkSession


### Set up the Spark session:


In [3]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Smart Building HVAC Monitoring") \
    .getOrCreate()


25/02/04 03:19:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/04 03:19:29 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Simulate sensor data:

Use Spark’s rate source to generate continuous readings from multiple rooms.


In [4]:
from pyspark.sql.functions import expr, rand,when

# Simulate sensor data with room IDs and readings
sensor_data = spark.readStream.format("rate").option("rowsPerSecond", 5).load() \
    .withColumn("room_id", expr("CAST(value % 10 AS STRING)")) \
    .withColumn("temperature", when(expr("value % 10 == 0"), 15)  # Set temperature to 15 for one specific record
                .otherwise(20 + rand() * 25)) \
    .withColumn("humidity", expr("40 + rand() * 30"))

### Create a temporary SQL view:

Create temporary SQL view to perform SQL queries on the streaming data.


In [5]:
# Create a temporary SQL view for the sensor data
sensor_data.createOrReplaceTempView("sensor_table")


### Define SQL queries for aggregation and analysis:

* **Critical temperature query**: Detect rooms with critical temperature levels
* **Average readings query**: Calculate average readings over a 1-minute window
* **Attention needed query**: Identify rooms that need immediate attention based on humidity levels


In [6]:
# SQL Query to detect rooms with critical temperatures
critical_temperature_query = """
    SELECT 
        room_id, 
        temperature, 
        humidity, 
        timestamp 
    FROM sensor_table 
    WHERE temperature < 18 OR temperature > 60
"""

# SQL Query to calculate average readings over a 1-minute window
average_readings_query = """
    SELECT 
        room_id, 
        AVG(temperature) AS avg_temperature, 
        AVG(humidity) AS avg_humidity, 
        window.start AS window_start 
    FROM sensor_table
    GROUP BY room_id, window(timestamp, '1 minute')
"""

# SQL Query to find rooms that need immediate attention based on humidity
attention_needed_query = """
    SELECT 
        room_id, 
        COUNT(*) AS critical_readings 
    FROM sensor_table 
    WHERE humidity < 45 OR humidity > 75
    GROUP BY room_id
"""


### Execute the SQL queries:

Execute each SQL query to create streaming DataFrames.


In [7]:
# Execute the critical temperature query
critical_temperatures_stream = spark.sql(critical_temperature_query)


# Execute the average readings query
average_readings_stream = spark.sql(average_readings_query)

# Execute the attention needed query
attention_needed_stream = spark.sql(attention_needed_query)






### Output the results to the console:

Display the results from each query in real-time.


In [8]:
# Output the results to the console for all queries
critical_query = critical_temperatures_stream.writeStream \
    .outputMode("append") \
    .format("console") \
    .queryName("Critical Temperatures") \
    .start()

average_query = average_readings_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Average Readings") \
    .start()

attention_query = attention_needed_stream.writeStream \
    .outputMode("complete") \
    .format("console") \
    .queryName("Attention Needed") \
    .start()



### Keep the streams running:

Ensure that the streaming queries continue to run to process incoming data.


In [9]:
# Keep the streams running

print("********Critical Temperature Values*******")
critical_query.awaitTermination()

print("********Average Readings Values********")
average_query.awaitTermination()
print("********Attention Needed Values********")
attention_query.awaitTermination()


********Critical Temperature Values*******
-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------+--------+---------+
|room_id|temperature|humidity|timestamp|
+-------+-----------+--------+---------+
+-------+-----------+--------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|48.000765526308555|2025-02-04 03:21:...|
|      0|       15.0|41.256482058914884|2025-02-04 03:21:...|
|      0|       15.0| 41.18665468552949|2025-02-04 03:21:...|
+-------+-----------+------------------+--------------------+



-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
+-------+-----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0|65.99242093372524|2025-02-04 03:21:...|
|      0|       15.0|47.81788419291462|2025-02-04 03:21:...|
|      0|       15.0|66.29212191724935|2025-02-04 03:21:...|
+-------+-----------+-----------------+--------------------+

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+---------------+------------+------------+
|room_id|avg_temperature|avg_humidity|window_start|
+-------+---------------+------------+------------+
+-------+---------------+------------+------------+



-------------------------------------------
Batch: 3
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 61.40859866483078|2025-02-04 03:21:...|
|      0|       15.0| 64.97046986318045|2025-02-04 03:21:...|
|      0|       15.0|46.664620120578675|2025-02-04 03:21:...|
|      0|       15.0| 57.76723959771104|2025-02-04 03:21:...|
|      0|       15.0| 53.33710969074489|2025-02-04 03:21:...|
|      0|       15.0|  48.9250957737628|2025-02-04 03:21:...|
|      0|       15.0| 43.30639032381356|2025-02-04 03:21:...|
|      0|       15.0| 42.33676191354539|2025-02-04 03:21:...|
|      0|       15.0| 57.93369991461472|2025-02-04 03:21:...|
|      0|       15.0| 56.55806245093525|2025-02-04 03:21:...|


...

-------------------------------------------
Batch: 11
-------------------------------------------
+-------+-----------+-----------------+--------------------+
|room_id|temperature|         humidity|           timestamp|
+-------+-----------+-----------------+--------------------+
|      0|       15.0|68.94441875880328|2025-02-04 03:23:...|
|      0|       15.0|   42.63843156735|2025-02-04 03:23:...|
|      0|       15.0|46.59402817816379|2025-02-04 03:23:...|
|      0|       15.0|  53.503549551471|2025-02-04 03:23:...|
|      0|       15.0|51.26956356143801|2025-02-04 03:23:...|
|      0|       15.0|51.50490449889431|2025-02-04 03:23:...|
+-------+-----------+-----------------+--------------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|               10|
|      3|               13|
|      8|               14|
|      0|                9

-------------------------------------------
Batch: 12
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|44.193466605592064|2025-02-04 03:23:...|
|      0|       15.0|52.724966782774565|2025-02-04 03:23:...|
|      0|       15.0| 66.36273646916237|2025-02-04 03:23:...|
|      0|       15.0| 44.25901812512393|2025-02-04 03:23:...|
|      0|       15.0| 50.61217211219925|2025-02-04 03:23:...|
|      0|       15.0|  40.8989063111894|2025-02-04 03:23:...|
+-------+-----------+------------------+--------------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       window_start|
+-------+------------------+------------------+-

-------------------------------------------
Batch: 13
-------------------------------------------
-------------------------------------------
Batch: 6
-------------------------------------------
+-------+-----------------+
|room_id|critical_readings|
+-------+-----------------+
|      7|               12|
|      3|               17|
|      8|               14|
|      0|               12|
|      5|               17|
|      6|               10|
|      9|                8|
|      1|               14|
|      4|               11|
|      2|               17|
+-------+-----------------+

+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0| 58.06781126438878|2025-02-04 03:23:...|
|      0|       15.0| 63.10421583147809|2025-02-04 03:24:...|
|      0|       15.0| 62.67189597941879|2025-02-04 03:24:...|
|      0|       15.0|40.724823584382705|

-------------------------------------------
Batch: 14
-------------------------------------------
-------------------------------------------
Batch: 6
-------------------------------------------
+-------+-----------+------------------+--------------------+
|room_id|temperature|          humidity|           timestamp|
+-------+-----------+------------------+--------------------+
|      0|       15.0|46.924183562496594|2025-02-04 03:24:...|
|      0|       15.0|  45.5481615608736|2025-02-04 03:24:...|
|      0|       15.0|42.450385858851725|2025-02-04 03:24:...|
|      0|       15.0| 51.01234159107403|2025-02-04 03:24:...|
|      0|       15.0| 66.31929938205484|2025-02-04 03:24:...|
|      0|       15.0| 48.13261499546658|2025-02-04 03:24:...|
|      0|       15.0|  50.7956088457893|2025-02-04 03:24:...|
+-------+-----------+------------------+--------------------+

+-------+------------------+------------------+-------------------+
|room_id|   avg_temperature|      avg_humidity|       

[Stage 43:=>                                                      (7 + 8) / 200]

### Conclusion
In this lab, you explored the use of Apache Spark in smart building monitoring, particularly focusing on HVAC (heating, ventilation, and air conditioning) systems. You now understand the Spark's distributed architecture. You also understand how to simulate real-time sensor data for temperature and humidity, execute SQL queries to identify critical environmental conditions, and output aggregated results for immediate insights.


## Author(s)

Lakshmi Holla

## Other Contributors
Malika Singla
